In [1]:
%matplotlib
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
from PIL import Image
from scipy import ndimage
from scipy import misc

from sklearn.datasets import fetch_mldata
from sklearn.neural_network import MLPRegressor

Using matplotlib backend: MacOSX


In [2]:
IMAGE_FOLDER_PATH = "/Users/zackakil/Desktop/capture clean/"

In [3]:
image_labels = pd.Series.from_csv('rugby_image_labels.csv')
image_labels = image_labels.dropna()

In [4]:
image_labels.keys()[0]

'1489359202.14.jpg'

In [5]:
output = {}

face = misc.imread(IMAGE_FOLDER_PATH+image_labels.keys()[0], flatten=True)
face1 = misc.imread(IMAGE_FOLDER_PATH+image_labels.keys()[1], flatten=True)     
plt.imshow((face[200:300] - face1[200:300])>30)
plt.show()

In [6]:
face1 = misc.imread(IMAGE_FOLDER_PATH+image_labels.keys()[1])     
plt.imshow(face/255)
plt.show()

In [9]:
f = get_image(IMAGE_FOLDER_PATH+image_labels.keys()[1])
plt.imshow(f)
plt.show()
# remove un-interesting bits of image
image_matrix[200:300]

NameError: name 'image_matrix' is not defined

In [10]:
plt.imshow(misc.imread(IMAGE_FOLDER_PATH+image_labels.keys()[0], flatten=True));
plt.show()

In [11]:
UPPER_FRAME = 300
LOWER_FRAME = 200
THRESHOLD = 30
def comapare_frames(frame_1_file_name, frame_2_file_name):
    f1 = misc.imread(frame_1_file_name, flatten=True)
    f2 = misc.imread(frame_2_file_name, flatten=True)
    diff = (f1[LOWER_FRAME:UPPER_FRAME] - f2[LOWER_FRAME:UPPER_FRAME])>THRESHOLD
    return [diff.sum(axis=0),diff]

def get_image(file_name):
    return misc.imread(file_name)

In [12]:
def process_images(image_labels, image_folder_name):
    '''
    Create lists of engneered image features.
    '''

    frame_compares_sum = []
    frame_compares = []
    frame_images = []
    i = 0
    images_to_process = image_labels[:]
    for index, value in images_to_process[:-1].iteritems():
        i += 1
        curr_image = image_folder_name + index
        next_image_index = image_labels.index.get_loc(index)+1
        next_image_key = image_folder_name + image_labels.keys()[next_image_index]
        out = comapare_frames(curr_image, next_image_key)
        frame_compares.append(out[1])
        frame_compares_sum.append(out[0])
        frame_images.append(get_image(next_image_key))
        if not (i % 25):
            print('{0} images processed'.format(i))
    return (frame_compares_sum, frame_compares, frame_images)

In [13]:
frame_compares_sum, frame_compares, frame_images = process_images(image_labels, IMAGE_FOLDER_PATH)

25 images processed
50 images processed
75 images processed
100 images processed
125 images processed
150 images processed
175 images processed
200 images processed
225 images processed
250 images processed
275 images processed
300 images processed
325 images processed
350 images processed
375 images processed
400 images processed
425 images processed
450 images processed
475 images processed
500 images processed
525 images processed
550 images processed
575 images processed
600 images processed
625 images processed
650 images processed
675 images processed
700 images processed
725 images processed
750 images processed
775 images processed
800 images processed
825 images processed
850 images processed
875 images processed
900 images processed
925 images processed
950 images processed
975 images processed
1000 images processed
1025 images processed
1050 images processed
1075 images processed
1100 images processed
1125 images processed
1150 images processed
1175 images processed
1200 ima

In [14]:
def process_images_from_folder(label_file_name, image_folder_name):

    _image_labels = pd.Series.from_csv(label_file_name)
    _image_labels = _image_labels.dropna()
    
    return process_images(_image_labels, image_folder_name)
    
    

In [15]:
image_labels_2 = pd.Series.from_csv('rugby_image_labels_2.csv')
image_labels_2 = image_labels_2.dropna()

In [16]:
folder = "/Users/zackakil/Desktop/capture/"
label_fname = 'rugby_image_labels_2.csv'
frame_compares_sum_2, frame_compares_2, frame_images_2 = process_images_from_folder(label_fname, folder)

25 images processed
50 images processed
75 images processed
100 images processed
125 images processed
150 images processed
175 images processed
200 images processed
225 images processed
250 images processed
275 images processed
300 images processed
325 images processed
350 images processed
375 images processed
400 images processed
425 images processed
450 images processed
475 images processed
500 images processed
525 images processed
550 images processed
575 images processed
600 images processed
625 images processed
650 images processed
675 images processed
700 images processed
725 images processed
750 images processed
775 images processed
800 images processed
825 images processed
850 images processed
875 images processed
900 images processed
925 images processed
950 images processed
975 images processed
1000 images processed
1025 images processed
1050 images processed
1075 images processed
1100 images processed
1125 images processed
1150 images processed
1175 images processed
1200 ima

In [17]:
def temporal_data(data, frames, end_to_end=True):
    '''
    Join frames of sequencial items of data together into idenvidual items.
    '''
    output = []
    
    # check if user wants frames to overlap or be end-to-end     
    if end_to_end:
        item_end_frames = frames
        len_adjust = 0
    else:
        item_end_frames = 1
        len_adjust = frames - 1

    for i in range(0, len(data) - len_adjust, item_end_frames):
        start_array = np.array([])
        for j in range(frames):
            start_array = np.concatenate((start_array, data[i+j]), axis=0)
        output.append(start_array)
    return output

In [18]:
tet = []
for i in range(20):
    tet.append(list(range(i, i+5)))
tet

[[0, 1, 2, 3, 4],
 [1, 2, 3, 4, 5],
 [2, 3, 4, 5, 6],
 [3, 4, 5, 6, 7],
 [4, 5, 6, 7, 8],
 [5, 6, 7, 8, 9],
 [6, 7, 8, 9, 10],
 [7, 8, 9, 10, 11],
 [8, 9, 10, 11, 12],
 [9, 10, 11, 12, 13],
 [10, 11, 12, 13, 14],
 [11, 12, 13, 14, 15],
 [12, 13, 14, 15, 16],
 [13, 14, 15, 16, 17],
 [14, 15, 16, 17, 18],
 [15, 16, 17, 18, 19],
 [16, 17, 18, 19, 20],
 [17, 18, 19, 20, 21],
 [18, 19, 20, 21, 22],
 [19, 20, 21, 22, 23]]

In [19]:
temporal_data(tet, 2, False)

[array([ 0.,  1.,  2.,  3.,  4.,  1.,  2.,  3.,  4.,  5.]),
 array([ 1.,  2.,  3.,  4.,  5.,  2.,  3.,  4.,  5.,  6.]),
 array([ 2.,  3.,  4.,  5.,  6.,  3.,  4.,  5.,  6.,  7.]),
 array([ 3.,  4.,  5.,  6.,  7.,  4.,  5.,  6.,  7.,  8.]),
 array([ 4.,  5.,  6.,  7.,  8.,  5.,  6.,  7.,  8.,  9.]),
 array([  5.,   6.,   7.,   8.,   9.,   6.,   7.,   8.,   9.,  10.]),
 array([  6.,   7.,   8.,   9.,  10.,   7.,   8.,   9.,  10.,  11.]),
 array([  7.,   8.,   9.,  10.,  11.,   8.,   9.,  10.,  11.,  12.]),
 array([  8.,   9.,  10.,  11.,  12.,   9.,  10.,  11.,  12.,  13.]),
 array([  9.,  10.,  11.,  12.,  13.,  10.,  11.,  12.,  13.,  14.]),
 array([ 10.,  11.,  12.,  13.,  14.,  11.,  12.,  13.,  14.,  15.]),
 array([ 11.,  12.,  13.,  14.,  15.,  12.,  13.,  14.,  15.,  16.]),
 array([ 12.,  13.,  14.,  15.,  16.,  13.,  14.,  15.,  16.,  17.]),
 array([ 13.,  14.,  15.,  16.,  17.,  14.,  15.,  16.,  17.,  18.]),
 array([ 14.,  15.,  16.,  17.,  18.,  15.,  16.,  17.,  18.,  19.]),


In [20]:
def bin_sum(data_array, bins):
    bin_size = int(len(data_array)/bins)
    return data_array.reshape(-1, bin_size).sum(axis=1)

In [21]:
bin_sum(frame_compares_sum[0],20)

array([11, 32, 44, 65, 70,  5, 21, 10,  0,  0,  8,  3,  9,  0,  0,  0,  0,
        0,  1,  0])

# Animated visualisation

In [22]:
fig, (ax1, ax2, ax3) = plt.subplots(3,1)

im = ax1.imshow(frame_images_2[0][LOWER_FRAME:UPPER_FRAME], animated=True)
im2 = ax2.imshow(frame_compares_2[0], animated=True)
to_plot = frame_compares_sum_2[10].reshape(-1, 10).sum(axis=1)
im3, = ax3.plot(to_plot)

line = [im, im2, im3]

def updatefig(i):
    line[0].set_array(frame_images_2[i][LOWER_FRAME:UPPER_FRAME])
    line[1].set_array(frame_compares_2[i])
    to_plot = frame_compares_sum_2[i].reshape(-1, 10).sum(axis=1)
    line[2].set_ydata(to_plot)   
    return line

ani = animation.FuncAnimation(fig, 
                              updatefig,
                              np.arange(200, 300),
                              interval=150,
                              blit=True)
plt.show()

In [85]:

plt.imshow( ((frame_images_2[131][LOWER_FRAME:UPPER_FRAME] - frame_images_2[132][LOWER_FRAME:UPPER_FRAME])**2)[:,:,0]>100 )
plt.grid(False)
plt.show()

In [47]:
fig, (ax1) = plt.subplots(1,1)

im = ax1.imshow(frame_images_2[0], animated=True)
# ax1.grid(False)
line = [im]

def updatefig(i):
    line[0].set_array(frame_images_2[i])
    return line

ani = animation.FuncAnimation(fig, 
                              updatefig,
                              np.arange(200, 300),
                              interval=150,
                              blit=True)
plt.show()

In [23]:
ani.save('norm_animation_10.gif', writer='imagemagick', fps=10)

In [24]:
frame_compares_sum[0].reshape(-1, 20).sum(axis=1)

array([ 0, 18, 25, 28, 50, 22, 52, 27,  5, 21,  0, 10,  0,  0,  0,  0,  0,
        8,  3,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0])

# Assemble training data

In [25]:
def bin_sum_and_normalise(sum_data, num_of_bins = 20, normalise_var = 100):
    _binned_pixel_diffs = [bin_sum(d, num_of_bins)/normalise_var for d in sum_data]
    return _binned_pixel_diffs

binned_pixel_diffs = bin_sum_and_normalise(frame_compares_sum)
binned_pixel_diffs[:2]

[array([ 0.11,  0.32,  0.44,  0.65,  0.7 ,  0.05,  0.21,  0.1 ,  0.  ,
         0.  ,  0.08,  0.03,  0.09,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.01,  0.  ]),
 array([ 0.  ,  0.55,  0.22,  0.85,  2.36,  1.75,  0.  ,  0.71,  0.01,
         0.  ,  0.12,  2.49,  2.35,  1.48,  0.3 ,  0.34,  0.  ,  0.01,
         0.  ,  0.  ])]

In [26]:
# temporal bundel multiple frames together into rows
num_of_frames_per_row = 2
temporaly_bundeled_data =  np.matrix(temporal_data(binned_pixel_diffs, num_of_frames_per_row, end_to_end=False))
temporaly_bundeled_data[:2]

matrix([[ 0.11,  0.32,  0.44,  0.65,  0.7 ,  0.05,  0.21,  0.1 ,  0.  ,
          0.  ,  0.08,  0.03,  0.09,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
          0.01,  0.  ,  0.  ,  0.55,  0.22,  0.85,  2.36,  1.75,  0.  ,
          0.71,  0.01,  0.  ,  0.12,  2.49,  2.35,  1.48,  0.3 ,  0.34,
          0.  ,  0.01,  0.  ,  0.  ],
        [ 0.  ,  0.55,  0.22,  0.85,  2.36,  1.75,  0.  ,  0.71,  0.01,
          0.  ,  0.12,  2.49,  2.35,  1.48,  0.3 ,  0.34,  0.  ,  0.01,
          0.  ,  0.  ,  0.  ,  0.2 ,  0.16,  0.22,  0.59,  0.35,  0.  ,
          0.  ,  0.  ,  0.  ,  0.09,  0.03,  0.02,  0.  ,  0.  ,  0.  ,
          0.  ,  0.  ,  0.  ,  0.  ]])

In [27]:
# gather desired output to match with last frame of bundeled data
# normilsie by the maximum pixel location (640 becuassee that is the width of the frame)
normalise_var = 640
desired_output_for_frames = image_labels.values[1:-1]/normalise_var
desired_output_for_frames[:3]

array([ 0.221875 ,  0.2484375,  0.24375  ])

In [28]:
def bundle_and_normalise(sum_data, image_labels, num_of_frames_per_row=2, normalise_var=640): 
    temporaly_bundeled_data =  np.matrix(temporal_data(sum_data, num_of_frames_per_row, end_to_end=False))
    desired_output_for_frames = image_labels.values[1:-1]/normalise_var
    return (temporaly_bundeled_data, desired_output_for_frames)

temporaly_bundeled_data, desired_output_for_frames = bundle_and_normalise(binned_pixel_diffs, image_labels)

desired_output_for_frames[:3], temporaly_bundeled_data[:4]

(array([ 0.221875 ,  0.2484375,  0.24375  ]),
 matrix([[ 0.11,  0.32,  0.44,  0.65,  0.7 ,  0.05,  0.21,  0.1 ,  0.  ,
           0.  ,  0.08,  0.03,  0.09,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
           0.01,  0.  ,  0.  ,  0.55,  0.22,  0.85,  2.36,  1.75,  0.  ,
           0.71,  0.01,  0.  ,  0.12,  2.49,  2.35,  1.48,  0.3 ,  0.34,
           0.  ,  0.01,  0.  ,  0.  ],
         [ 0.  ,  0.55,  0.22,  0.85,  2.36,  1.75,  0.  ,  0.71,  0.01,
           0.  ,  0.12,  2.49,  2.35,  1.48,  0.3 ,  0.34,  0.  ,  0.01,
           0.  ,  0.  ,  0.  ,  0.2 ,  0.16,  0.22,  0.59,  0.35,  0.  ,
           0.  ,  0.  ,  0.  ,  0.09,  0.03,  0.02,  0.  ,  0.  ,  0.  ,
           0.  ,  0.  ,  0.  ,  0.  ],
         [ 0.  ,  0.2 ,  0.16,  0.22,  0.59,  0.35,  0.  ,  0.  ,  0.  ,
           0.  ,  0.09,  0.03,  0.02,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
           0.  ,  0.  ,  0.  ,  0.01,  0.15,  0.61,  0.56,  0.29,  0.  ,
           0.  ,  0.  ,  0.  ,  0.08,  0.11,  0.06,  0.  ,  0.  ,  0.  ,


In [29]:
len(desired_output_for_frames), len(temporaly_bundeled_data)

(2532, 2532)

In [30]:
def transofrm_data_into_input_data_format(pixel_sum_data, image_labels):
    _binned_pixel_diffs = bin_sum_and_normalise(pixel_sum_data)
    return bundle_and_normalise(_binned_pixel_diffs, image_labels)

In [31]:
i_1, o_1 = transofrm_data_into_input_data_format(frame_compares_sum, image_labels)
i_2, o_2 = transofrm_data_into_input_data_format(frame_compares_sum_2, image_labels_2)

In [32]:
all_inputs = np.concatenate((i_1, i_2))
all_inputs.shape

(5358, 40)

In [33]:
all_outputs = np.concatenate((o_1, o_2))
all_outputs.shape

(5358,)

# Train neural network 

In [34]:
from sklearn.model_selection import train_test_split

In [35]:

# mnist = fetch_mldata("MNIST original")
# rescale the data, use the traditional train/test split
# X, y = temporaly_bundeled_data, desired_output_for_frames
X, y = all_inputs, all_outputs
split_boundary = 2000
X_train, X_test = X[:split_boundary], X[split_boundary:]
y_train, y_test = y[:split_boundary], y[split_boundary:]


In [36]:
X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(
                        X, y, test_size=0.2, random_state=42)

In [37]:

train_results = []
test_results = []
layer_vals = list(range(1,10,1))
for i in layer_vals:
    X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(
                        X, y, test_size=1 - (i/10), random_state=42)
    rgr = MLPRegressor(hidden_layer_sizes=(25), max_iter=2000, alpha=15,
                        solver='lbfgs', verbose=10, tol=1e-4, random_state=1,
                        learning_rate_init=.1)

    rgr.fit(X_train_t, y_train_t)
    train_score = rgr.score(X_train_t, y_train_t)
    test_score = rgr.score(X_test_t, y_test_t)
    print("Training set score: %f" % train_score)
    print("Test set score: %f" % test_score)
    train_results.append(train_score)
    test_results.append(test_score)
    print(1 - (i/10))

Training set score: 0.723692
Test set score: 0.614937
0.9
Training set score: 0.768655
Test set score: 0.710480
0.8
Training set score: 0.781157
Test set score: 0.748301
0.7
Training set score: 0.798824
Test set score: 0.754357
0.6
Training set score: 0.808120
Test set score: 0.769145
0.5
Training set score: 0.812376
Test set score: 0.771438
0.4
Training set score: 0.813952
Test set score: 0.778593
0.30000000000000004
Training set score: 0.813058
Test set score: 0.790092
0.19999999999999996
Training set score: 0.823484
Test set score: 0.789914
0.09999999999999998


In [38]:
plt.plot(train_results, c ='g')
plt.plot(test_results, c ='r')
plt.show()

In [39]:
X_train_t[:,:20].shape

(4822, 20)

In [170]:
rgr = MLPRegressor(hidden_layer_sizes=(23), 
                   max_iter=1, alpha=10,
                   solver='lbfgs', 
                   verbose=10, 
                   tol=1e-10, 
                   random_state=1,
                   learning_rate_init=.0001, 
                   warm_start= True)

# rgr.fit(X_train_t, y_train_t)
# train_score = rgr.score(X_train_t, y_train_t)
# test_score = rgr.score(X_test_t, y_test_t)
# print("Training set score: %f" % train_score)
# print("Test set score: %f" % test_score)

In [171]:
nn_test_score = []
heur_test_score = []

for i in range(500):
    rgr.fit(X_train_t, y_train_t)
    nn_test_score.append(rgr.score(X_test_t, y_test_t))
    heur_test_score.append(r2_score(y_test_t,heur_predict(X_test_t)))

In [196]:
plt.title('Regessor NN vs Heuristic model')
plt.ylabel('R^2 Score')
plt.xlabel('Learning iteration')
nn_plot = plt.plot(nn_test_score, c='g', label='Regressor Neural Net')
heur_plot = plt.plot(heur_test_score, c='b', label='Heuristic model')
plt.ylim(0.4,0.9)
plt.xlim(0,500)

plt.legend(handles=[nn_plot[0], heur_plot[0]],loc=4)

In [181]:
plt.plot(layer_vals,train_results)
plt.plot(layer_vals,test_results)

In [116]:
o

array([ 0.41887759,  0.393201  ,  0.3339116 ,  0.30326352,  0.23129711,
        0.2233137 ,  0.20990473,  0.21598249,  0.23328139,  0.22389982,
        0.22539453,  0.21228117,  0.2168425 ,  0.24404686,  0.30652133,
        0.32857684,  0.3303517 ,  0.37438198,  0.34383299,  0.35259473,
        0.42068756,  0.40130302,  0.30868615,  0.26787117,  0.27707563,
        0.2898766 ,  0.37866319,  0.4896563 ,  0.46138087,  0.43612926,
        0.36054929,  0.35682666,  0.31865374,  0.24613009,  0.24985167,
        0.22875137,  0.22349348,  0.27487995,  0.39021832,  0.42267581,
        0.39063639,  0.44364757,  0.45282155,  0.4794069 ,  0.44244302,
        0.4719319 ,  0.42557487,  0.34528786,  0.34881111,  0.32351177,
        0.30407746,  0.28643083,  0.28127448,  0.32255585,  0.33178714,
        0.28999655,  0.33717701,  0.354685  ,  0.382536  ,  0.3766554 ,
        0.35927514,  0.42203991,  0.42062053,  0.4783876 ,  0.55776537,
        0.55929178,  0.60349442,  0.6345226 ,  0.72244151,  0.62

In [101]:
o = rgr.predict(all_inputs[:1000])
plt.plot(o, c='r')
plt.plot(all_outputs[:1000], c='g')
plt.plot(heur[:1000])

In [130]:
np.absolute(o - all_outputs[:1000]).sum()

77.54140757795534

In [133]:
np.absolute(heur[:1000].reshape(1,-1) - all_outputs[:1000]).sum()

93.739474643392185

In [135]:
from sklearn.metrics import r2_score

In [202]:
r2_score(all_outputs[:1000],heur[:1000])

0.69987925443047627

In [137]:
r2_score(all_outputs[:1000],o)

0.7935131908142703

# Test against heuristic model

In [63]:
bar = np.array(list(range(20))).reshape(1,20)
np.multiply(all_inputs[0,:20],bar).sum()/ all_inputs[0,:20].sum()

3.7813620071684588

In [75]:
all_inputs[0,:20].sum()

2.79

In [90]:
(all_inputs[:5,:20]*bar.reshape(20,1))/all_inputs[:5,:20].sum(axis=1)

matrix([[ 3.78136201],
        [ 8.29246677],
        [ 4.07228916],
        [ 4.77368421],
        [ 4.51598174]])

In [87]:
all_inputs[:5,20:]*bar.reshape(20,1)

matrix([[ 112.28],
        [   6.76],
        [   9.07],
        [   9.89],
        [  10.23]])

In [74]:
np.multiply(all_inputs[0,:20],bar).sum()

10.550000000000001

In [70]:
plt.plot(list(range(20)), all_inputs[0,:20].reshape(20,-1))

In [67]:
all_inputs[0,:20].reshape(20,-1)

matrix([[ 0.11],
        [ 0.32],
        [ 0.44],
        [ 0.65],
        [ 0.7 ],
        [ 0.05],
        [ 0.21],
        [ 0.1 ],
        [ 0.  ],
        [ 0.  ],
        [ 0.08],
        [ 0.03],
        [ 0.09],
        [ 0.  ],
        [ 0.  ],
        [ 0.  ],
        [ 0.  ],
        [ 0.  ],
        [ 0.01],
        [ 0.  ]])

In [203]:

def heur_predict(data):
    bar = np.array(list(range(20))).reshape(1,20)
    count_data = data[:,:20]*bar.reshape(20,1)
    sum_divider = data[:,:20].sum(axis=1)
    return (count_data/sum_divider)/19
    

In [201]:
heur = heur_predict(all_inputs)

In [95]:
plt.plot(heur_predict(all_inputs))

## Save model

In [40]:
from sklearn.externals import joblib

In [42]:
joblib.dump(rgr, 'rugby_tracker/rugby_predictor_v2.pkl', protocol=2) 

['rugby_tracker/rugby_predictor_v2.pkl']